In [1]:
import dotenv
import os
dotenv.load_dotenv()
from openai import AsyncOpenAI

client = AsyncOpenAI()
model = "gpt-4-1106-preview"

In [2]:
async def recommend_eval(messages, client):

  content = """
    You will be given the transcript of a therapy session. You are to generate four concise recommendations for the therapist to talk about next.
    Generate a json with the following format:
    {
      "recommendations": ["List", "of", "four", "recommendations"]
    }
    """

  messages = [
      {"role": "system", "content": content},
      {'role': 'user', 'content': messages}
  ]

  response = await client.chat.completions.create(
    model=model,
    response_format={ "type": "json_object" },
    messages=messages,
    temperature=0
  )

  return response.choices[0].message.content

async def mood_eval(messages, client):

  content = """
    You will be given the transcript of what a patient told a therapist during a therapy session. 
    Generate a json with the following format:
    {
      "mood": "one word to describe their current mood"
    }
    """

  messages = [
      {"role": "system", "content": content},
      {'role': 'user', 'content': messages}
  ]

  response = await client.chat.completions.create(
    model=model,
    response_format={ "type": "json_object" },
    messages=messages,
    temperature=0
  )

  return response.choices[0].message.content

def process_conversation_client_only(excerpts):
  messages = ""

  for excerpt in excerpts:
    if excerpt['role'] == "patient":
      messages += excerpt['content'] + "\n"
      
  return messages
      
def process_conversation(excerpts):
  messages = ""
  
  for excerpt in excerpts:
    messages += excerpt['role'] + ": " + excerpt['content'] + "\n"
    
  return messages









In [3]:
mock = [
  {"role": "therapist", "content": "Good morning, how are you feeling today?"},
  {"role": "patient", "content": "Honestly, I've been feeling a bit overwhelmed lately."},
  {"role": "therapist", "content": "I'm sorry to hear that. Can you tell me a bit more about what's been overwhelming you?"},
  {"role": "patient", "content": "Work has been extremely stressful, and it's been hard to balance that with family life."},
  {"role": "therapist", "content": "Balancing work and family can certainly be challenging. How have you been coping with this stress?"},
  {"role": "patient", "content": "I try to relax in the evenings, but I'm finding it hard to switch off from work."}
]

In [4]:
mock_patient = process_conversation_client_only(mock)
mock_transcript = process_conversation(mock)
print(mock_patient)
print(mock_transcript)

Honestly, I've been feeling a bit overwhelmed lately.
Work has been extremely stressful, and it's been hard to balance that with family life.
I try to relax in the evenings, but I'm finding it hard to switch off from work.

therapist: Good morning, how are you feeling today?
patient: Honestly, I've been feeling a bit overwhelmed lately.
therapist: I'm sorry to hear that. Can you tell me a bit more about what's been overwhelming you?
patient: Work has been extremely stressful, and it's been hard to balance that with family life.
therapist: Balancing work and family can certainly be challenging. How have you been coping with this stress?
patient: I try to relax in the evenings, but I'm finding it hard to switch off from work.



In [5]:
recommend_profile = await recommend_eval(mock_transcript, client)
print(recommend_profile)
mood_profile = await mood_eval(mock_patient, client)
print(mood_profile)


{
  "recommendations": [
    "Discuss time management and prioritization strategies to help balance work and family life.",
    "Explore relaxation techniques that can help the patient switch off from work in the evenings.",
    "Consider setting boundaries at work to reduce stress and prevent burnout.",
    "Talk about the importance of self-care and finding personal time amidst a busy schedule."
  ]
}
{
  "mood": "overwhelmed"
}


In [8]:
import json
print(json.loads(recommend_profile))

{'recommendations': ['Discuss time management and prioritization strategies to help balance work and family life.', 'Explore relaxation techniques that can help the patient switch off from work in the evenings.', 'Consider setting boundaries at work to reduce stress and prevent burnout.', 'Talk about the importance of self-care and finding personal time amidst a busy schedule.']}
